In [1]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, Subset  # [변경됨]: random_split 대신 Subset 사용
import json
import pandas as pd
import numpy as np  # [변경됨]: numpy 임포트


# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 데이터 경로 설정
data_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\data\brand_data5"
processed_data_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\data\processed_data5"
json_save_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\json"
csv_save_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\csv"
model_save_path = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\modelnew\resnet18-5.pth"

# JSON, CSV 저장 폴더 생성
os.makedirs(json_save_dir, exist_ok=True)
os.makedirs(csv_save_dir, exist_ok=True)

# 모든 브랜드 폴더를 순회하면서 하위 제품 폴더 이동
for brand in os.listdir(data_dir):
    brand_path = os.path.join(data_dir, brand)
    if os.path.isdir(brand_path):
        for product in os.listdir(brand_path):
            product_path = os.path.join(brand_path, product)
            if os.path.isdir(product_path):
                new_product_path = os.path.join(processed_data_dir, product)
                os.makedirs(new_product_path, exist_ok=True)
                for image in os.listdir(product_path):
                    src = os.path.join(product_path, image)
                    dst = os.path.join(new_product_path, image)
                    shutil.copy(src, dst)

print("✅ 모든 제품 폴더를 'processed_data5'로 이동 완료!")

# 이미지 변환 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드
dataset = datasets.ImageFolder(root=processed_data_dir, transform=transform)
num_classes = len(dataset.classes)
print(f"총 클래스 개수: {num_classes}")
print("클래스 목록:", dataset.classes)

# [변경됨] 데이터셋 분할 - 클래스별로 7:2:1 비율로 분할
# 각 클래스별 인덱스 추출
class_indices = {cls: [] for cls in range(num_classes)}
for idx, (path, label) in enumerate(dataset.imgs):
    class_indices[label].append(idx)

train_indices, val_indices, test_indices = [], [], []
for label, indices in class_indices.items():
    np.random.shuffle(indices)  # 인덱스 랜덤 셔플
    n = len(indices)
    n_train = int(0.7 * n)
    n_val = int(0.2 * n)
    # 나머지는 테스트셋에 할당
    train_indices.extend(indices[:n_train])
    val_indices.extend(indices[n_train:n_train+n_val])
    test_indices.extend(indices[n_train+n_val:])

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)
# [변경됨 끝]

# JSON 파일로 데이터셋 저장
def save_json(subset_dataset, dataset_name):
    data_list = [{"image_path": subset_dataset.dataset.imgs[i][0], "label": int(subset_dataset.dataset.imgs[i][1])} 
                for i in subset_dataset.indices]
    json_path = os.path.join(json_save_dir, f"{dataset_name}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data_list, f, indent=4)
    print(f"✅ {dataset_name}.json 저장 완료!")

save_json(train_dataset, "train5")
save_json(val_dataset, "val5")
save_json(test_dataset, "test5")

# 데이터 로더 설정
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print("✅ 데이터 로더 생성 완료!")

# 모델 설정 (ResNet18)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)

# [추가됨] Learning rate scheduler (ReduceLROnPlateau 사용 - 검증 정확도를 기준으로 학습률 조절)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, verbose=True)

# 학습 기록 저장용 DataFrame
history = {"epoch": [], "train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

# [추가됨] Early Stopping 설정 변수 (개선이 없을 시 조기 종료)
early_stop_patience = 20
no_improve_count = 0

# [추가됨] 로그 파일 생성 (학습 기록 추가 저장)
log_file = os.path.join(csv_save_dir, "train_log5.txt")
with open(log_file, "w") as lf:
    lf.write("Epoch,Train Loss,Train Acc,Val Loss,Val Acc,Learning Rate\n")

# 모델 학습 함수
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    global no_improve_count  # [추가됨] early stopping 카운트 변수 사용
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            correct_train += predicted.eq(labels).sum().item()
            total_train += labels.size(0)

        train_acc = 100 * correct_train / total_train
        train_loss /= len(train_loader)

        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                correct_val += predicted.eq(labels).sum().item()
                total_val += labels.size(0)

        val_acc = 100 * correct_val / total_val
        val_loss /= len(val_loader)

        history["epoch"].append(epoch + 1)
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"🚀 Epoch [{epoch+1}/{num_epochs}] | 📉 Train Loss: {train_loss:.4f} | 🎯 Train Accuracy: {train_acc:.4f} | 📑 Valid Loss: {val_loss:.4f} | 🎯 Valid Accuracy: {val_acc:.4f}")

        # [추가됨] Learning rate scheduler step (검증 정확도를 기준으로 학습률 조정)
        scheduler.step(val_acc)
        current_lr = optimizer.param_groups[0]['lr']
        print(f"현재 학습률: {current_lr:.6f}")

        # [추가됨] 로그 파일에 학습 기록 추가 저장
        with open(log_file, "a") as lf:
            lf.write(f"{epoch+1},{train_loss:.4f},{train_acc:.4f},{val_loss:.4f},{val_acc:.4f},{current_lr:.6f}\n")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), model_save_path)
            print(f"📌 새로운 최고 모델 저장됨! (Epoch {epoch+1}, Accuracy: {val_acc:.4f}) ✅")
            no_improve_count = 0  # [추가됨] 개선되었으므로 카운트 초기화
        else:
            no_improve_count += 1  # [추가됨] 개선되지 않음

        # [추가됨] Early Stopping 조건 확인 (개선이 없으면 조기 종료)
        if no_improve_count >= early_stop_patience:
            print(f"🚨 조기 종료: {early_stop_patience} 에폭 동안 개선 없음.")
            break

    print(f"🎯 최종 최고 검증 정확도: {best_acc:.2f}%")

# 모델 학습 실행
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=500)

# 학습 과정 CSV로 저장
df = pd.DataFrame(history)
csv_path = os.path.join(csv_save_dir, "텐트분류3.csv")
df.to_csv(csv_path, index=False)
print(f"✅ 학습 기록 저장 완료: {csv_path}")

# 최적 모델 로드 후 테스트 데이터 평가
model.load_state_dict(torch.load(model_save_path))
model.eval()
correct_test, total_test = 0, 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        correct_test += predicted.eq(labels).sum().item()
        total_test += labels.size(0)

test_acc = 100 * correct_test / total_test
print(f"🎯 테스트 데이터셋 정확도: {test_acc:.2f}%")


Using device: cuda
✅ 모든 제품 폴더를 'processed_data5'로 이동 완료!
총 클래스 개수: 171
클래스 목록: ['12X', '17.2', '4S 와이드', '6.3', 'A5', 'A7', 'A9', 'D1', 'IE', 'M1', 'P1', 'T 코어', 'U.L', 'X 코어', 'X5', 'protect', '가마보코', '고스트', '고틀랜드', '골드키위', '구아바', '그랑베르크', '그랜즈빌 렉타 타프', '나르시스 돔', '나마츠', '날로', '네뷸라', '네스트', '노나돔', '니악', '데크쉘터', '도크돔', '라이더스', '라인스피엘', '랜드록', '랜드마크', '랜드브리즈', '레이사', '렉타', '렉타 타프', '렉타타프', '로맨틱터틀', '루나', '리빙쉘', '링스틴드', '마리나', '멀티펑션 암막타프', '메쉬 타프스크린', '몽가', '문라이트', '미라클스크린', '미라클패밀리', '바랑에르', '바랑에르돔', '발할', '뱅가드', '보니아또', '보스', '볼트', '브이타프', '블랙 코트', '비무르', '비바 렉타 타프', '비바돔', '비티호른', '빅', '빌리지', '새턴 쉘터', '새턴 에어텐트', '새턴2룸', '세이랜드', '셰어', '솔로', '쉘터G', '쉘터돔', '슈퍼팰리스', '스타이카', '스텔라릿지', '썬블럭', '아고라', '아리에스', '아스가르드', '아스터돔', '아스트라돔', '아케디아', '아퀼라', '아틀라스', '아틀란틱', '아티카', '알락', '알베르게', '알비온', '알파 룸', '알파인돔', '알파인라이트', '알페임', '야른비드', '어메니티돔', '에볼루션 메쉬쉘터', '에어 도킹', '에어 리빙 쉘터', '에코 소울 예이예이', '옐로우스톤', '오우치', '오토듀얼팔레스', '오토하우스', '오트라', '오파러스', '오프랜드', '와가야노', '와이즈 타프쉘', '와일드 필드 오스카', '와일드 필드 옥타곤', '

c:\Users\user\anaconda3\envs\classification3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\classification3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


🚀 Epoch [1/500] | 📉 Train Loss: 3.3700 | 🎯 Train Accuracy: 34.2060 | 📑 Valid Loss: 2.2753 | 🎯 Valid Accuracy: 53.6799
현재 학습률: 0.000100
📌 새로운 최고 모델 저장됨! (Epoch 1, Accuracy: 53.6799) ✅
🚀 Epoch [2/500] | 📉 Train Loss: 1.9650 | 🎯 Train Accuracy: 60.7665 | 📑 Valid Loss: 1.7908 | 🎯 Valid Accuracy: 62.0189
현재 학습률: 0.000100
📌 새로운 최고 모델 저장됨! (Epoch 2, Accuracy: 62.0189) ✅
🚀 Epoch [3/500] | 📉 Train Loss: 1.4303 | 🎯 Train Accuracy: 70.2428 | 📑 Valid Loss: 1.5358 | 🎯 Valid Accuracy: 66.2728
현재 학습률: 0.000100
📌 새로운 최고 모델 저장됨! (Epoch 3, Accuracy: 66.2728) ✅
🚀 Epoch [4/500] | 📉 Train Loss: 1.1046 | 🎯 Train Accuracy: 75.7067 | 📑 Valid Loss: 1.4429 | 🎯 Valid Accuracy: 67.4882
현재 학습률: 0.000100
📌 새로운 최고 모델 저장됨! (Epoch 4, Accuracy: 67.4882) ✅
🚀 Epoch [5/500] | 📉 Train Loss: 0.8820 | 🎯 Train Accuracy: 80.0133 | 📑 Valid Loss: 1.4026 | 🎯 Valid Accuracy: 68.6361
현재 학습률: 0.000100
📌 새로운 최고 모델 저장됨! (Epoch 5, Accuracy: 68.6361) ✅
🚀 Epoch [6/500] | 📉 Train Loss: 0.7096 | 🎯 Train Accuracy: 83.0393 | 📑 Valid Loss: 1.